# Зеркальный спуск

# Целевая функция

Нижний индекс - номер числка/вектора в наборе.

$$ x, s, a_i \in \mathbb{R}_+^n;\quad c \in \mathbb{R}_+^m; \quad i=\overline{1,m} $$

$$ f(x) = s^T x - \sum_{i=1}^m c_i \log(a_i^T x) + h(x) $$

$$ \nabla f(x) = s + \sum_{i=1}^m \dfrac{c_i}{a_i^T x}a_i + \nabla h(x) $$

В общем виде алгоритм зеркального спуска (верхний индекс - номер итерации):

$$ x^{k+1} = \arg \min_{x \in \mathbb{R}_+^n} ~ \left\langle \alpha^k \nabla f(x^k), x \right\rangle + V_{x^k}(x) $$

Где градиент исследуемой функции $f(x)$:

$$ \nabla f(x^k) = \alpha^k\left(s + \sum_{i=1}^m \dfrac{c_i}{a_i^T x^k}a_i + \nabla h(x^k)\right)$$

В нашей задачи для оценки сходимости используется первая норма, поэтому в качестве расстояния Брэгмана мы выберем 

$$ V_{x}(y) = \sum_{i=1}^n x_i \log \dfrac{y_i}{x_i}$$

Итого, переходя к нормированным переменным $\tilde{x}\in\Delta_n$ (после переходя со звёздочкой волну над $x$ опустим):

$$ x^{k+1} = \arg \min_{x \in \mathbb{R}_+^n}  \left\langle \alpha^k \nabla f(x^k), x \right\rangle + \sum_{i=1}^n x^k_i \log \dfrac{x_i}{x^k_i} =^* $$

$$ =^*  \arg \min_{x \in \Delta_n} \left\langle \alpha^k \nabla f(x^k), x \right\rangle + \sum_{i=1}^n x^k_i \log \dfrac{x_i}{x^k_i} = $$

$$ = x^k \cdot \dfrac{\exp\left(-\alpha^k \nabla f(x^k) \right)}{\left\lVert x^k\cdot \exp\left(-\alpha^k \nabla f(x^k)\right) \right\rVert_1}$$

In [70]:
def f(x, s, c, a, h):
    # h - penalty function
    # s,c,a - const (see problem statement)
    return np.dot(s, x) + sum(c[:] * np.log(np.dot(a[:][:], x))) + h(x)

def grad_f(x, s, c, a, grad_h):
    # grad_h - penalty function gradient
    # s,c,a - const (see problem statement)
    return s + sum(c[:] / np.dot(a[:], x) * a[:]) + grad_h(x)

class MirrorDescend:
    
    dim   = 0     # dimension
    grad  = None  # objective function gradient
    alpha = None  # MD coef

    def __init__(self, _dim, _grad, _alpha):
        self.dim   = _dim
        self.alpha = _alpha
        self.grad  = _grad
    
    def do_nothing(x, t):
        return
    
    def run(self, x0, steps_num = 1000, user_action = do_nothing):
        x      = np.zeros(self.dim)
        next_x = np.zeros(self.dim)
        
        x = x0
        user_action(x, 0)

        for i in range(1, steps_num):
            z = np.exp(- self.alpha(i) * self.grad(x))
            next_x = x * z / np.linalg.norm(x * z, 1)
            user_action(next_x, i)
            
            x, next_x = next_x, x # swap array references

        return x

In [71]:
def h_test(x):
    return 0.0

def grad_h_test(x):
    g = np.zeros(len(x))
    return g

def alpha_test(k):
    return 0.001

x0 = [1.0, 2.0, 3.0]
s  = [1.0, 0.0, 0.0]
c  = [1.0, -1.0, 0.0]
a  = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]

def grad_test(x):
    return grad_f(x, s, c, a, grad_h_test)

md = MirrorDescend(len(x0), grad_test, alpha_test)
md.run(x0, 100)

array([0.08914119, 0.42557429, 0.48528452])

In [10]:
# Common seed functions

import numpy as np

def sqNorm(x):
    return 0.5 * (x ** 2)

def V_sqNorm(x):
    return 0.5 * (x - y) ** 2

def shanEntropy(x):
    return x * np.log(x) - x

def bitEntropy(x):
    return x * np.log(x) + (1 - x) * np.log(1 - x)

def burgEntropy(x):
    return -np.log(x)

def hellinger(x):
    return -((1 - x ** 2) ** 0.5)

def lpQuasiNorm(p, x):
    if p <= 0 or p >= 1:
        raise ValueError("p = " + str(p) + " is outside of (0,1) bounds in function lpQuasiNorm")
    return - (x ** p)

def lpNorm(p, x):
    if p <= 1: # or p > inf
        raise ValueError("p = " + str(p) + " is outside of (1,inf) bounds in function lpNorm")
    return np.abs(x) ** p

def exponential(x):
    return np.exp(x)

def inverse(x):
    return 1 / x